In [44]:
from platform import python_version
python_version()

'3.7.4'

In [36]:
from __future__ import print_function, division
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import ticker
import nltk
import seaborn as sns
import pandas as pd
import scipy as sp
from scipy.stats import entropy
from collections import Counter, defaultdict
import networkx as nx

import pylotwhale.utils.whaleFileProcessing as fp
import pylotwhale.utils.fileCollections as fcll
import pylotwhale.utils.annotationTools as annT
import pylotwhale.NLP.annotations_analyser as aa
import pylotwhale.NLP.ngramO_beta as ngr

import pylotwhale.utils.dataTools as daT
import pylotwhale.utils.plotTools as pT

import pylotwhale.NLP.myStatistics_beta as mysts
import pylotwhale.NLP.tempoTools as tT
import pylotwhale.utils.netTools as nT
import librosa

In [63]:
def search_sequence_numpy(arr, seq):
    """ Find sequence in an array

    Parameters
    ----------
    arr: input 1D array
    seq: input 1D array

    Return
    ------
    Output : 1D Array of indices in the input array that satisfy the
        matching of input sequence in the input array.
        In case of no match, empty array is returned.
    """

    # Store sizes of input array and sequence
    Na, Nseq = arr.size, seq.size

    # Range of sequence
    r_seq = np.arange(Nseq)

    # Create 2D array of sliding indices across entire length of input array.
    # Match up with the input sequence & get the matching starting indices.
    M = (arr[np.arange(Na - Nseq + 1)[:, None] + r_seq] == seq).all(1)

    # Get the range of those indices as final output
    if M.any:

        return np.where(np.convolve(M, np.ones((Nseq), dtype=int)) > 0)[0]
    else:
        return np.array([])

In [64]:
arr = np.array(list('abcbdababz'))
seq = np.array([list('ab')])
np.testing.assert_equal(search_sequence_numpy(arr, seq), 
                        np.array([0, 1, 5, 6, 7, 8]))

In [43]:
t0=1
tf=3
y, sr = librosa.core.load(fi, offset=t0,
                              sr=None, duration=tf-t0)

In [8]:
#pDir = os.path.dirname(os.path.abspath(__file__))

df_file = '../pylotwhale/tests/data/groupB_annotations_df.csv'
# load
df = pd.read_csv(df_file)#; df= df0

tapedf = daT.dictOfGroupedDataFrames(df)

In [19]:
print(tapedf.keys())
thisdf = tapedf[113]
thisdf.head()

dict_keys([113, 114, 115, 111])


,call,t0,tf,ioi,ici,cl,file_name,tape
0,126i,10.220170,10.555920,7.798030,7.462280,0.335750,WAV_0113_001-48kHz-hv.txt,113
1,126i,18.018200,18.389919,8.369591,7.997872,0.371719,WAV_0113_001-48kHz-hv.txt,113
2,126i,26.387791,26.995331,3.037679,2.430139,0.607540,WAV_0113_001-48kHz-hv.txt,113
3,126i,29.425470,29.957060,1.750660,1.219070,0.531590,WAV_0113_001-48kHz-hv.txt,113
4,126i,31.176130,31.623791,2.162340,1.714679,0.447661,WAV_0113_001-48kHz-hv.txt,113


In [24]:
float('-inf') 

-inf

In [25]:
def df2listOfSeqs(df, Dt=None, l='call', time_param='ict_end_start', time_param_dtype = float):
    '''
    returns the sequences of <l>, chains of elements separated by the interval Dt
    Parameters:
    -----------
        df : dataframe, must have a time column (time_param)
        time_param : name of the sequence definition time, default "ict"
        l : specifies the sequence type. default "calls"
            l can also be a list eg. ['call', 'ict'] but takes more time
            in such case a list of tuple-sequences will be returned
        Dt : time inteval for sequence definition, default (-infty, 0.5)
    creates a list with the sequences (lists)
    '''
    if isinstance(l, str):
        fun = lambda x: x
    else:
        fun = lambda x: tuple(x)
 
    if Dt is None: Dt = (float('-inf') , 0.5)

    ict = df[time_param].values

    if np.isnan(ict[-1]):  # ict end with nan
        endix = None
    elif isinstance(ict[-1], time_param_dtype): # ends with float?
        endix = -1
    else:
        assert False, 'wrong dtype'

    seqsLi = []  # initialise list of sequences
    subLi = [fun(df[l].iloc[0])]  # initialise sequence w/ first element

    for i in range(len(ict))[:endix]:
        if Dt[0] <= ict[i] <= Dt[1]:  # is part of the sequence?
            subLi.append(df[l].iloc[i+1])
        elif ict[i] >= Dt[1]:  # nope, then start a new sequence
            seqsLi.append(subLi)  # save the one we had
            subLi = [fun(df[l].iloc[i+1])]  # start new sequence

    seqsLi.append(subLi)  # append last sequence

    return seqsLi    


In [30]:
l='call'
thisdf[l].iloc[2]

'126i'

In [31]:
s = df2listOfSeqs(thisdf, l='call', time_param='ici', Dt=(-1,1))

-1 7.462280000000001 1
-1 7.997872 1
-1 2.430139 1
-1 1.21907 1
-1 1.714679 1
-1 4.500561 1
-1 4.556511 1
-1 1.05119 1
-1 8.851681 1
-1 7.487808 1
-1 0.41021 1
-1 0.669693 1
-1 2.538071 1
-1 1.934341 1
-1 -0.11346400000000001 1
-1 3.157593 1
-1 0.391777 1
-1 -0.322762 1
-1 4.353775 1
-1 3.101441 1
-1 -0.659836 1
-1 3.5013199999999998 1
-1 6.824997 1
-1 11.517708 1
-1 3.9011 1
-1 1.506905 1
-1 -0.669395 1
-1 0.302498 1
-1 2.0784990000000003 1
-1 0.771401 1
-1 4.716401 1
-1 1.274994 1
-1 3.8250889999999997 1
-1 16.643203 1
-1 4.232803 1
-1 0.13589400000000001 1
-1 5.779495 1
-1 9.428803 1
-1 0.787399 1
-1 0.8272860000000001 1
-1 1.0752110000000001 1
-1 0.8394010000000001 1
-1 14.508896 1
-1 10.711808 1
-1 32.86679 1
-1 6.097306 1
-1 0.8912959999999999 1
-1 1.9225009999999998 1
-1 2.5261080000000002 1
-1 0.027999999999999997 1
-1 0.047898 1
-1 0.071899 1
-1 7.666108 1
-1 2.6480099999999998 1
-1 12.474487 1
-1 5.453796 1
-1 0.036011 1
-1 -0.075897 1
-1 16.819122 1
-1 -0.13430799999999998 1

In [32]:
s

[['126i'],
 ['126i'],
 ['126i'],
 ['126i'],
 ['126i'],
 ['126i'],
 ['126i'],
 ['126i'],
 ['126i'],
 ['126ii'],
 ['127', '133', '126ii'],
 ['126ii'],
 ['133', '131ii'],
 ['126i', '093ii', '133'],
 ['126i'],
 ['126i', '133'],
 ['126i'],
 ['126ii'],
 ['131ii'],
 ['093ii'],
 ['133', '131iii', '133'],
 ['133', '126i'],
 ['126ii'],
 ['134'],
 ['133'],
 ['126i'],
 ['093ii', '126ii'],
 ['133'],
 ['126ii', '093ii', '133'],
 ['126ii', '133'],
 ['126ii'],
 ['133'],
 ['126i'],
 ['127', '126i'],
 ['131iii'],
 ['131i', '129', '129', '129'],
 ['126i'],
 ['126i'],
 ['126i'],
 ['132A', '132B', '126i'],
 ['131iii', '126i'],
 ['127'],
 ['126i'],
 ['126i', '126i'],
 ['131iii', '130', '129', '126i', '129', '129'],
 ['128i', '127', '130', '131iii', '129', '129', '129', '129'],
 ['128i', '129', '127', '129', '129', '133', '131iii', '130'],
 ['128i', '128i', '129', '129'],
 ['131iii'],
 ['131iii'],
 ['131iii', '129', '129', '103'],
 ['103'],
 ['093ii'],
 ['103'],
 ['103'],
 ['131iii', '129', '129', '103'],
 [

In [9]:
## parameters and settings
Dt = 0.3; Dtint = (None, Dt)
## define the sequences
sequencesList0 = aa.dfDict2listOfSeqs(tapedf, Dt=Dtint, l='call', time_param='ici')

TypeError: '>=' not supported between instances of 'float' and 'NoneType'

In [ ]:


# # Load df

# In[3]:


# tape separation
tapedf = daT.dictOfGroupedDataFrames(df)

def test_data():
    # N_calls
    assert(len(df) == 425)
    # N_call types
    assert(len(set(df['call'])) == 22)
    # tapes set
    assert(set(df['tape'].values) ==  set([113, 114, 115, 111]))
    assert(set(tapedf.keys()) == set([113, 114, 115, 111]))


# # Bigrams  and randomisations test
# 
# Define the **sequences**

# In[4]:


## parameters and settings
Dt = 0.3; Dtint = (None, Dt)
## define the sequences
sequencesList0 = aa.dfDict2listOfSeqs(tapedf, Dt=Dtint, l='call', time_param='ici')
## filter out isolated calls (sequences of size one)
sequencesList = [l for l in sequencesList0 if len(l)>1]

seqO = mysts.sequenceBigrams(sequencesList)

def test_seqO():
    assert(len(seqO.seqOfSeqs) == 71)
    assert(seqO.callCounts['129'] == 89)


# **Bigrams**, counts and probabilities

# In[5]:


## arange order of the calls
minCalls = 5
condsLi = seqO.conditionsLi(minCalls)
sampsLi = seqO.samplesLi(minCalls)
print("conds", condsLi, '\nsamps', sampsLi)

## filter out bigrams observed less than 5 times
minBigrams=3
mask = seqO.df_cfd.loc[condsLi, sampsLi] < minBigrams #sys.float_info.min

def test_conds_samps():
    assert(np.shape(mask) == (len(condsLi), len(sampsLi)))


# ### Randomise 
# 
# randomise
# 
# <!---
# #### less than minCalls
# mask_p_value = p_values > pc
# mask_minCalls = M <= minCalls
# mask = np.logical_or(mask_p_value, mask_minCalls)# mask_minCalls #mask_p_value# 
# 
# f,ax = plt.subplots(1,3, figsize=(18, 4))
# sns.heatmap(mask_minCalls, ax=ax[0])
# sns.heatmap(mask_p_value, ax=ax[1])
# sns.heatmap(mask, ax=ax[2])
# 
# !--->

# In[21]:


Nsh = 200
np.random.seed(0)

p_values, sh_dists = mysts.randtest4bigrmas_inSequences(seqO.seqOfSeqs, Nsh,
                                                        condsLi=condsLi, sampsLi=sampsLi)


# In[22]:


def test_p_values():
    np.testing.assert_approx_equal(114.93, np.sum(p_values.values), significant=3)
    np.testing.assert_approx_equal(p_values.loc['129', '129'], 0)
    np.testing.assert_approx_equal(p_values.loc['130', '130'], 0)
    assert(p_values.loc['130', '129'] > 0.2)
    


# ### Network

# In[23]:


pc = 0.01
bigrams_pc = daT.get_indexColFromDataFrame(p_values, lambda x: x > pc)

kkD_counts = seqO.df_cfd
minBigrams = 3
bigrams_minBigrams = daT.get_indexColFromDataFrame(seqO.df_cfd, lambda x: x < minBigrams)

rmBigrams = bigrams_pc | bigrams_minBigrams

netO = nT.dict2network(seqO.cfd)


# In[24]:


def test_network():
    assert(len(bigrams_pc) == 133)
    assert(len(bigrams_minBigrams) == 374)
    assert(len(rmBigrams) == 386)
    assert(netO.net_dict['079'] == {'126i': 1, '129': 1})

